In [1020]:
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn import decomposition
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
import warnings
from sklearn.ensemble import BaggingClassifier 
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import glob
pd.set_option("max_columns", None)

In [1069]:
def data_proc(loc):
    dire = glob.glob(loc[0] + "/*.csv")
    tmp = pd.read_csv(dire[0])
    df = pd.DataFrame(columns = tmp.columns)
    df['label'] = 0
    count=0
    for dir in loc:
        dir2 = glob.glob(dir + "/*.csv")
        for i in range(len(dir2)):
            tmp = pd.read_csv(dir2[i])
            #dropping first 7 seconds of the data
            tmp.drop(tmp.loc[(tmp['Time'] < tmp['Time'][0]+7)].index, axis = 0, inplace=True)
            if "Alone" in dir2[i]:
                tmp['label'] = 0
            elif "Spontan" in dir2[i]:
                tmp['label'] = 1
                #tmp =unite(tmp)
            elif "Sync" in dir2[i]:
                tmp['label'] = 2
                #tmp = unite(tmp)
            #df = df.loc[:,df.culomns.duplicated()]
            df = (pd.concat([df,tmp]))
    df[" Hand Type"].replace({"left": 0, "right": 1}, inplace=True)
    return df

def alone(loc):
    right = pd.read_csv(r"C:\Users\USER\Desktop\קבצים לעבודת גמר\extraFiles/HandRight.csv")
    right[" Hand Type"].replace({"left": 0, "right": 1}, inplace=True)
    dire = glob.glob(loc[0] + "/*.csv")
    tmp = pd.read_csv(dire[0])
    df = pd.DataFrame(columns = tmp.columns)
    df['label'] = 0
    for dir in loc:
        dir2 = glob.glob(dir + "/*.csv")
        for i in range(len(dir2)):
            if "Alone" in dir2[i]:
                tmp['label'] = 0  
                tmp = pd.read_csv(dir2[i])
                #dropping first 7 seconds of the data
                tmp.drop(tmp.loc[(tmp['Time'] < tmp['Time'][0]+7)].index, axis = 0, inplace=True)
                tmp['label'] = 0
                tmp[" Hand Type"].replace({"left": 0, "right": 1}, inplace=True)
                df = (pd.concat([df,tmp]))
                df = pd.merge(tmp, right,left_index=True, right_index=True, how='outer')
                #pd.concat([df, df2], axis=1)

    df = df.dropna()
    return df

Reading the data and combinning left and right hands

In [1070]:
dire = glob.glob(r'C:\Users\USER\Desktop\Training/*')
not_aln = data_proc(dire)

#deleting the data with one hand
t = not_aln.loc[(not_aln[' # hands'] == 1)].index
not_aln.drop(t ,axis = 0, inplace=True)
left = not_aln.loc[not_aln[' Hand Type'] == 0]
right = not_aln.loc[not_aln[' Hand Type'] == 1]
not_aln = pd.merge(left, right, on=[' Frame ID','Time'], how='outer')
not_aln = not_aln.dropna()
not_aln

,Time,Frame ID,Hand Type_x,# hands_x,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,Wrist Pos X_x,Wrist Pos Y_x,Wrist Pos Z_x,Elbow pos X_x,Elbow Pos Y_x,Elbow Pos Z_x,Grab Strenth_x,Grab Angle_x,Pinch Strength_x,label_x,Hand Type_y,# hands_y,Position X_y,Position Y_y,Position Z_y,Velocity X_y,Velocity Y_y,Velocity Z_y,Pitch_y,Roll_y,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y,label_y
1,377.3322,133320.0,0.0,2,-90.76891,80.19047,4.998824,56.923870,-153.98010,-96.907950,1.174060,0.884395,1.060637,-135.1231,27.50848,19.933540,-261.4986,-182.59940,68.75103,0.0,0.360957,0.0,1.0,1.0,2,115.1082,181.1342,-1.425699,-55.76400,-389.44110,51.12590,0.355496,-0.243332,-0.478105,155.4285,159.3211,51.79894,321.7448,14.100630,169.05580,0.0,0.489229,0.0,1.0
2,377.3488,133322.0,0.0,2,-89.50469,77.57844,3.803055,75.782650,-131.06480,-66.235170,1.155368,0.915238,1.001529,-131.3373,23.44940,20.533460,-241.8985,-193.79370,75.97872,0.0,0.397707,0.0,1.0,1.0,2,114.5054,174.7521,-0.133119,-39.77689,-369.92380,85.97456,0.336554,-0.231727,-0.485765,155.2746,154.1053,53.19535,320.7865,10.178190,173.12560,0.0,0.430351,0.0,1.0
3,377.3653,133324.0,0.0,2,-87.74129,76.18815,2.513577,95.718640,-74.18849,-73.287850,1.129648,0.928391,0.955936,-128.3818,21.78429,21.082360,-231.5567,-196.50430,85.80962,0.0,0.325913,0.0,1.0,1.0,2,113.8313,169.4297,0.768792,-39.06313,-293.45450,51.95496,0.334020,-0.236116,-0.483134,154.5182,148.9042,54.21458,320.5969,6.302337,174.90910,0.0,0.462008,0.0,1.0
4,377.3819,133326.0,0.0,2,-87.43260,76.28526,1.508446,4.581604,22.20399,-50.192670,1.115047,0.930319,0.931579,-127.6079,21.96202,21.257110,-229.1942,-194.55490,93.92790,0.0,0.406301,0.0,1.0,1.0,2,113.0777,164.5829,1.586354,-42.97567,-275.99310,51.41563,0.340320,-0.228040,-0.478630,153.4810,143.7801,55.14198,319.6718,1.455538,175.99200,0.0,0.517912,0.0,1.0
5,377.3985,133327.0,0.0,2,-87.57170,76.62472,1.234697,-16.314690,39.81437,-32.107480,1.109929,0.931101,0.926407,-127.7505,22.43810,21.349370,-230.0657,-192.46970,97.67774,0.0,0.447324,0.0,1.0,1.0,2,112.5172,162.1344,1.970408,-65.72982,-287.19000,45.04501,0.340913,-0.218023,-0.479577,152.9387,141.3940,55.53011,319.1259,-0.884198,176.43720,0.0,0.521856,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25874,365.9342,42710.0,0.0,2,-128.54170,151.26810,-58.056940,56.927480,-411.99820,36.522750,0.544789,1.769163,0.401752,-158.4455,112.56210,-8.386236,-360.1552,50.91410,125.02330,0.0,0.072727,0.0,2.0,1.0,2,102.9425,198.3317,-45.138960,57.69714,90.79050,44.17404,1.453731,-1.177701,-1.285025,134.3051,136.0471,-40.71001,244.6106,-59.681180,67.95710,0.0,0.000000,0.0,2.0
25875,365.9509,42712.0,0.0,2,-127.88370,144.42220,-57.221520,39.527260,-409.53660,57.872170,0.512069,1.795802,0.393831,-157.6622,107.35880,-6.255749,-359.2655,50.74594,129.52530,0.0,0.102246,0.0,2.0,1.0,2,104.2880,200.4675,-44.087910,62.83340,115.48050,73.52728,1.502334,-1.170726,-1.406168,135.8917,138.2810,-42.35088,234.8518,-64.252450,64.75884,0.0,0.000000,0.0,2.0
25876,365.9673,42714.0,0.0,2,-127.28110,137.64710,-55.658230,60.419170,-402.94580,99.212360,0.491071,1.811933,0.393753,-157.1180,101.79940,-3.860802,-356.9034,50.44975,136.61840,0.0,0.102073,0.0,2.0,1.0,2,104.8781,201.8934,-42.092990,30.95871,105.11820,116.19240,1.562618,-1.181244,-1.550808,136.0406,139.5943,-43.75756,226.5645,-68.983850,59.13079,0.0,0.000000,0.0,2.0
25877,365.9842,42716.0,0.0,2,-126.16660,130.84610,-54.101770,93.572760,-388.12690,62.157910,0.482603,1.835022,0.385263,-155.3881,95.57278,-1.611514,-352.0472,50.69477,145.34620,0.0,0.095909,0.0,2.0,1.0,2,105.8573,203.8846,-40.742390,38.90388,73.38232,75.83213,1.629559,-1.190103,-1.712266,137.2473,142.0417,-45.92920,226.1846,-67.838140,55.68773,0.0,0.000000,0.0,2.0


In [1071]:
aln = alone(dire)
aln.head()

,Time_x,Frame ID_x,Hand Type_x,# hands_x,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,Wrist Pos X_x,Wrist Pos Y_x,Wrist Pos Z_x,Elbow pos X_x,Elbow Pos Y_x,Elbow Pos Z_x,Grab Strenth_x,Grab Angle_x,Pinch Strength_x,label,Time_y,Frame ID_y,Hand Type_y,# hands_y,Position X_y,Position Y_y,Position Z_y,Velocity X_y,Velocity Y_y,Velocity Z_y,Pitch_y,Roll_y,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
210,696.6509,54136.0,0.0,1.0,-89.64678,209.6048,46.58355,254.074400,-261.213400,-112.89250,1.422460,1.421660,0.941405,-109.7800,140.0193,48.77377,-261.2395,-66.16656,82.03649,0.0,0.0,0.0,0.0,131.7869,15882,1,1,16.808010,201.1241,1.858396,-536.5381,-95.47218,-55.603570,0.182805,0.024803,-0.147946,39.60963,187.4913,72.53979,178.9622,33.41478,245.2226,0.0,0.497389,0.0
211,696.6837,54140.0,0.0,1.0,-81.27094,199.3336,41.74398,238.226600,-327.093800,-152.85690,1.385698,1.375930,0.965133,-105.1037,131.0047,46.36422,-260.5015,-70.53613,88.69434,0.0,0.0,0.0,0.0,131.8035,15884,1,1,7.717093,198.7942,1.322116,-511.3952,-143.18630,-25.353340,0.182737,0.038054,-0.154041,31.09795,185.3946,71.86711,176.2882,34.29144,242.3996,0.0,0.413895,0.0
212,696.7185,54144.0,0.0,1.0,-76.00336,193.9522,38.37697,87.297390,-10.381170,-71.99138,1.369062,1.333171,1.002113,-102.5148,126.6451,44.09771,-256.2060,-74.77591,92.75569,0.0,0.0,0.0,0.0,131.8202,15886,1,1,-0.685181,196.0823,0.434851,-476.4807,-149.04750,-57.567310,0.201434,0.038249,-0.157297,23.08376,181.6445,70.70461,173.5955,33.60156,239.3430,0.0,0.350033,0.0
213,696.7516,54148.0,0.0,1.0,-75.04813,193.7479,37.35566,-5.169657,10.255670,-22.80904,1.366562,1.321865,1.021135,-102.2178,126.6868,43.16080,-252.9394,-76.77298,92.61942,0.0,0.0,0.0,0.0,131.8371,15888,1,1,-8.004724,193.7121,0.295862,-401.3970,-151.58710,-12.232210,0.193186,0.048867,-0.163873,16.35544,180.0536,70.52497,171.7019,36.00371,238.2770,0.0,0.265133,0.0
214,696.7845,54152.0,0.0,1.0,-76.04221,194.0001,35.96521,-34.201280,-5.889267,-45.07218,1.352656,1.332684,0.990696,-103.0065,126.8914,42.43633,-253.5724,-77.46542,88.55816,0.0,0.0,0.0,0.0,131.8536,15890,1,1,-14.349020,191.3762,-0.068918,-350.9143,-134.33980,-3.816512,0.184482,0.055214,-0.170380,10.57506,178.5172,70.13129,169.8845,38.83430,237.9022,0.0,0.219063,0.0


In [1072]:
not_aln.drop([" Hand Type_y"," # hands_y","label_y"], axis=1, inplace=True)
not_aln = not_aln.rename(columns={'label_x': 'label', ' Frame ID_x':' Frame ID'})


In [1073]:
not_aln

,Time,Frame ID,Hand Type_x,# hands_x,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,Wrist Pos X_x,Wrist Pos Y_x,Wrist Pos Z_x,Elbow pos X_x,Elbow Pos Y_x,Elbow Pos Z_x,Grab Strenth_x,Grab Angle_x,Pinch Strength_x,label,Position X_y,Position Y_y,Position Z_y,Velocity X_y,Velocity Y_y,Velocity Z_y,Pitch_y,Roll_y,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
1,377.3322,133320.0,0.0,2,-90.76891,80.19047,4.998824,56.923870,-153.98010,-96.907950,1.174060,0.884395,1.060637,-135.1231,27.50848,19.933540,-261.4986,-182.59940,68.75103,0.0,0.360957,0.0,1.0,115.1082,181.1342,-1.425699,-55.76400,-389.44110,51.12590,0.355496,-0.243332,-0.478105,155.4285,159.3211,51.79894,321.7448,14.100630,169.05580,0.0,0.489229,0.0
2,377.3488,133322.0,0.0,2,-89.50469,77.57844,3.803055,75.782650,-131.06480,-66.235170,1.155368,0.915238,1.001529,-131.3373,23.44940,20.533460,-241.8985,-193.79370,75.97872,0.0,0.397707,0.0,1.0,114.5054,174.7521,-0.133119,-39.77689,-369.92380,85.97456,0.336554,-0.231727,-0.485765,155.2746,154.1053,53.19535,320.7865,10.178190,173.12560,0.0,0.430351,0.0
3,377.3653,133324.0,0.0,2,-87.74129,76.18815,2.513577,95.718640,-74.18849,-73.287850,1.129648,0.928391,0.955936,-128.3818,21.78429,21.082360,-231.5567,-196.50430,85.80962,0.0,0.325913,0.0,1.0,113.8313,169.4297,0.768792,-39.06313,-293.45450,51.95496,0.334020,-0.236116,-0.483134,154.5182,148.9042,54.21458,320.5969,6.302337,174.90910,0.0,0.462008,0.0
4,377.3819,133326.0,0.0,2,-87.43260,76.28526,1.508446,4.581604,22.20399,-50.192670,1.115047,0.930319,0.931579,-127.6079,21.96202,21.257110,-229.1942,-194.55490,93.92790,0.0,0.406301,0.0,1.0,113.0777,164.5829,1.586354,-42.97567,-275.99310,51.41563,0.340320,-0.228040,-0.478630,153.4810,143.7801,55.14198,319.6718,1.455538,175.99200,0.0,0.517912,0.0
5,377.3985,133327.0,0.0,2,-87.57170,76.62472,1.234697,-16.314690,39.81437,-32.107480,1.109929,0.931101,0.926407,-127.7505,22.43810,21.349370,-230.0657,-192.46970,97.67774,0.0,0.447324,0.0,1.0,112.5172,162.1344,1.970408,-65.72982,-287.19000,45.04501,0.340913,-0.218023,-0.479577,152.9387,141.3940,55.53011,319.1259,-0.884198,176.43720,0.0,0.521856,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25874,365.9342,42710.0,0.0,2,-128.54170,151.26810,-58.056940,56.927480,-411.99820,36.522750,0.544789,1.769163,0.401752,-158.4455,112.56210,-8.386236,-360.1552,50.91410,125.02330,0.0,0.072727,0.0,2.0,102.9425,198.3317,-45.138960,57.69714,90.79050,44.17404,1.453731,-1.177701,-1.285025,134.3051,136.0471,-40.71001,244.6106,-59.681180,67.95710,0.0,0.000000,0.0
25875,365.9509,42712.0,0.0,2,-127.88370,144.42220,-57.221520,39.527260,-409.53660,57.872170,0.512069,1.795802,0.393831,-157.6622,107.35880,-6.255749,-359.2655,50.74594,129.52530,0.0,0.102246,0.0,2.0,104.2880,200.4675,-44.087910,62.83340,115.48050,73.52728,1.502334,-1.170726,-1.406168,135.8917,138.2810,-42.35088,234.8518,-64.252450,64.75884,0.0,0.000000,0.0
25876,365.9673,42714.0,0.0,2,-127.28110,137.64710,-55.658230,60.419170,-402.94580,99.212360,0.491071,1.811933,0.393753,-157.1180,101.79940,-3.860802,-356.9034,50.44975,136.61840,0.0,0.102073,0.0,2.0,104.8781,201.8934,-42.092990,30.95871,105.11820,116.19240,1.562618,-1.181244,-1.550808,136.0406,139.5943,-43.75756,226.5645,-68.983850,59.13079,0.0,0.000000,0.0
25877,365.9842,42716.0,0.0,2,-126.16660,130.84610,-54.101770,93.572760,-388.12690,62.157910,0.482603,1.835022,0.385263,-155.3881,95.57278,-1.611514,-352.0472,50.69477,145.34620,0.0,0.095909,0.0,2.0,105.8573,203.8846,-40.742390,38.90388,73.38232,75.83213,1.629559,-1.190103,-1.712266,137.2473,142.0417,-45.92920,226.1846,-67.838140,55.68773,0.0,0.000000,0.0


In [1074]:
aln = aln.rename(columns={'Time_x': 'Time', ' Frame ID_x':' Frame ID'})
aln.drop([" # hands_y", "Time_y"," Frame ID_y"," Hand Type_y"], axis=1, inplace=True)
aln

,Time,Frame ID,Hand Type_x,# hands_x,Position X_x,Position Y_x,Position Z_x,Velocity X_x,Velocity Y_x,Velocity Z_x,Pitch_x,Roll_x,Yaw_x,Wrist Pos X_x,Wrist Pos Y_x,Wrist Pos Z_x,Elbow pos X_x,Elbow Pos Y_x,Elbow Pos Z_x,Grab Strenth_x,Grab Angle_x,Pinch Strength_x,label,Position X_y,Position Y_y,Position Z_y,Velocity X_y,Velocity Y_y,Velocity Z_y,Pitch_y,Roll_y,Yaw_y,Wrist Pos X_y,Wrist Pos Y_y,Wrist Pos Z_y,Elbow pos X_y,Elbow Pos Y_y,Elbow Pos Z_y,Grab Strenth_y,Grab Angle_y,Pinch Strength_y
210,696.6509,54136.0,0.0,1.0,-89.64678,209.6048,46.58355,254.074400,-261.213400,-112.892500,1.422460,1.421660,0.941405,-109.7800,140.0193,48.77377,-261.2395,-66.16656,82.03649,0.000000,0.000000,0.000000,0.0,16.808010,201.1241,1.858396,-536.5381,-95.47218,-55.603570,0.182805,0.024803,-0.147946,39.609630,187.4913,72.53979,178.9622,33.41478,245.2226,0.0,0.497389,0.0
211,696.6837,54140.0,0.0,1.0,-81.27094,199.3336,41.74398,238.226600,-327.093800,-152.856900,1.385698,1.375930,0.965133,-105.1037,131.0047,46.36422,-260.5015,-70.53613,88.69434,0.000000,0.000000,0.000000,0.0,7.717093,198.7942,1.322116,-511.3952,-143.18630,-25.353340,0.182737,0.038054,-0.154041,31.097950,185.3946,71.86711,176.2882,34.29144,242.3996,0.0,0.413895,0.0
212,696.7185,54144.0,0.0,1.0,-76.00336,193.9522,38.37697,87.297390,-10.381170,-71.991380,1.369062,1.333171,1.002113,-102.5148,126.6451,44.09771,-256.2060,-74.77591,92.75569,0.000000,0.000000,0.000000,0.0,-0.685181,196.0823,0.434851,-476.4807,-149.04750,-57.567310,0.201434,0.038249,-0.157297,23.083760,181.6445,70.70461,173.5955,33.60156,239.3430,0.0,0.350033,0.0
213,696.7516,54148.0,0.0,1.0,-75.04813,193.7479,37.35566,-5.169657,10.255670,-22.809040,1.366562,1.321865,1.021135,-102.2178,126.6868,43.16080,-252.9394,-76.77298,92.61942,0.000000,0.000000,0.000000,0.0,-8.004724,193.7121,0.295862,-401.3970,-151.58710,-12.232210,0.193186,0.048867,-0.163873,16.355440,180.0536,70.52497,171.7019,36.00371,238.2770,0.0,0.265133,0.0
214,696.7845,54152.0,0.0,1.0,-76.04221,194.0001,35.96521,-34.201280,-5.889267,-45.072180,1.352656,1.332684,0.990696,-103.0065,126.8914,42.43633,-253.5724,-77.46542,88.55816,0.000000,0.000000,0.000000,0.0,-14.349020,191.3762,-0.068918,-350.9143,-134.33980,-3.816512,0.184482,0.055214,-0.170380,10.575060,178.5172,70.13129,169.8845,38.83430,237.9022,0.0,0.219063,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2245,764.4578,59984.0,0.0,1.0,-125.81370,214.8894,20.59324,20.170870,19.997410,37.082170,1.099869,1.391745,0.409534,-146.6314,148.2971,44.40766,-258.0887,-77.52971,112.97060,0.116068,1.901876,0.000000,0.0,-11.488350,156.5860,48.362690,-145.6763,483.15760,192.147700,0.052809,0.044457,-0.042415,2.911529,155.4418,124.52100,102.1572,96.35275,374.2091,0.0,0.520259,0.0
2246,764.4913,59986.0,0.0,1.0,-125.92320,215.6769,22.09264,-6.867068,21.957800,42.485630,1.100429,1.404892,0.399021,-146.1113,148.8045,45.68930,-253.6847,-80.13625,109.99670,0.134497,1.952747,0.000000,0.0,-13.892390,164.0521,51.710350,-149.5128,406.57340,211.284700,0.077692,0.072020,-0.050988,1.291821,161.2216,127.65400,109.0853,96.04669,372.2403,0.0,0.499646,0.0
2247,764.5253,59988.0,0.0,1.0,-125.62170,215.7428,23.39750,17.454730,-3.217659,31.446690,1.101742,1.429256,0.379251,-144.8174,148.4588,46.66075,-249.6531,-83.33176,105.00430,0.120680,1.984825,0.000000,0.0,-16.512220,170.2372,55.295090,-155.2818,363.51390,226.139700,0.108562,0.081309,-0.057499,-0.565474,165.0822,130.93670,116.3977,91.35533,368.7994,0.0,0.574757,0.0
2248,764.5587,59989.0,0.0,1.0,-125.13520,215.6353,23.80248,25.567410,-5.648659,21.284490,1.107025,1.437351,0.368866,-143.8482,148.1024,46.72861,-248.2768,-84.26936,103.46830,0.097735,1.988376,0.000000,0.0,-18.654460,175.8734,59.295840,-121.0561,262.37540,216.857500,0.133075,0.094642,-0.065663,-1.870492,168.6951,134.53000,123.8105,82.91599,363.7611,0.0,0.629732,0.0


In [1077]:
train_df = pd.concat([aln,not_aln])
train_df.isna().sum()

Time                 0
 Frame ID            0
 Hand Type_x         0
 # hands_x           0
 Position X_x        0
 Position Y_x        0
 Position Z_x        0
 Velocity X_x        0
 Velocity Y_x        0
 Velocity Z_x        0
 Pitch_x             0
 Roll_x              0
 Yaw_x               0
 Wrist Pos X_x       0
 Wrist Pos Y_x       0
 Wrist Pos Z_x       0
 Elbow pos X_x       0
 Elbow Pos Y_x       0
 Elbow Pos Z_x       0
 Grab Strenth_x      0
 Grab Angle_x        0
 Pinch Strength_x    0
label                0
 Position X_y        0
 Position Y_y        0
 Position Z_y        0
 Velocity X_y        0
 Velocity Y_y        0
 Velocity Z_y        0
 Pitch_y             0
 Roll_y              0
 Yaw_y               0
 Wrist Pos X_y       0
 Wrist Pos Y_y       0
 Wrist Pos Z_y       0
 Elbow pos X_y       0
 Elbow Pos Y_y       0
 Elbow Pos Z_y       0
 Grab Strenth_y      0
 Grab Angle_y        0
 Pinch Strength_y    0
dtype: int64